# Imports

In [ ]:
import sys
sys.path.append('../')
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)

import spacy
from spacy.lang.en import English
from spacy.attrs import POS
nlp = spacy.load('en_core_web_lg')

In [ ]:
from nlp_utils import get_feats, lemmatize_doc
from gensim.models.tfidfmodel import TfidfModel
from gensim.corpora import Dictionary
from gensim.matutils import sparse2full

TODO: do something more principled here...

In [ ]:
def stop (t) :
    return t.is_stop or t.lemma_ in ['person', 'tangrams', 'tangram', 'look', 'like', 'tap', 'choose', 'zzz', 'xxx', 'yyy', 'pick', 'guy', 'blue', 'box']

# pre-process text by lemmatizing

In [ ]:
d_raw = pd.read_csv('../deidentified/combined.csv', encoding='latin-1')

In [ ]:
d_raw['text'] = [nlp(text) for text in d_raw['utterance']]
d_raw['non_stop_text'] = [[token for token in text if not token.is_stop] for text in d_raw['text']]

In [ ]:
d_raw['lemmatized_nonstop'] = [lemmatize_doc(parsed_text) for parsed_text in d_raw['non_stop_text']]

In [ ]:
d_raw.head()

In [ ]:
d = d_raw.copy()
gameidList = pd.unique(d.subid.ravel()).tolist()
tangramList = pd.unique(d.target.ravel()).tolist()

### Look at where conventions were introduced

The first-order case is where you just look at how often final words occurred at each previous round (percentage).

Then we can look at *first* round for each particular word... 

Then we can check who was speaker on that first round.

TODO: check cases where the 'matcher' on a round may have introduced a word

TODO: Split on PMI distribution as more systematic way of finding stop words... 

TODO: look at P(r + 1 | r) for local 'stickiness'

TODO: potentially add back in messages sent after selection

In [ ]:
rows = []

def stop (t) :
    return t.is_stop or t.lemma_ in ['person', 'look', 'like', 'tap', 'choose', 'zzz', 'xxx', 'yyy', 'pick', 'guy', 'blue', 'box']

# For each game, look at referring expressions produced by director on later round
for name, df in d.query('role == "director"').groupby(['subid', 'rep_num']) :
    for i, row in df.sort_values('target').reset_index().iterrows() :
        later_rep = row['rep_num']
        target = row['target']
        content_words = np.unique(
            [t.lemma_ for t in row.text 
             if t.pos_ in ["NOUN", "ADJ", 'VERB'] 
             and not stop(t)]
        )
        query_str = 'target == "{}"'.format(target)
        for j, word in enumerate(content_words) :
            for earlier_rep in range(1, later_rep) :
                earlier_df = d.query('rep_num == {} and role == "director" and subid == "{}"'
                                     .format(earlier_rep, name[0])).sort_values('target').reset_index()
                match = word in np.array(list(earlier_df.query(query_str)['lemmas'])).flatten()
                rows.append([row['experiment'], row['subid'], row['target'], row['person'], row['age'], earlier_rep, later_rep, word, match])

In [ ]:
words_df = pd.DataFrame(rows,
    columns = ['experiment', 'subid', 'target', 'final_round_person', 'age',
               'earlier_rep', 'later_rep',  'word', 'match']
)

In [ ]:
words_df.to_csv('../deidentified/word_matches.csv', index=False)

We can also look at the inverse: probability of words on current round appearing at end... 

In [ ]:
rows = []

# For each game, look at referring expressions produced by director on final round
for name, rep_df in d.query('role == "director"').groupby(['subid', 'rep_num']) :
    rep_df = rep_df.sort_values('target').reset_index()
    final_df = d.query('rep_num == 4 and role == "director" and subid == "{}"'.format(name[0])).sort_values('target').reset_index()
    
    # For each word used with each tangram, check whether it occured in each earlier round
    for i, row in rep_df.iterrows() :
        target = row['target']
        content_words = [t.lemma_ for t in row.text 
                         if t.pos_ in ["NOUN", "ADJ", 'VERB'] 
                         and not stop(t)]
        print('content', content_words)
        query_str = 'target == "{}"'.format(target)
        print(np.array(list(final_df.query(query_str)['lemmas'])).flatten())
        for j, word in enumerate(content_words) :
            final_match = word in np.array(list(final_df.query(query_str)['lemmas'])).flatten()
            rows.append([row['experiment'], row['subid'], row['rep_num'], row['target'], row['person'], row['age'], word, final_match])

In [ ]:
words_df = pd.DataFrame(rows,
    columns = ['experiment', 'subid', 'rep_num', 'target', 'person', 'age', 'word', 'final_match']
)
words_df.to_csv('../deidentified/inverse_word_matches.csv')

# Examine semantic embeddings
We'd like to pull out bag of words embeddings from NPs in each utterance in the cued dataset and cluster them for each tangram; expect to see different pairs in different parts of the space (i.e. to compute a d' for an 'idiosyncracy' or 'multiple equilibria' result) and also different utterances from single games closer together. 

In [ ]:
d_raw = (pd.read_csv('../data/deidentified/tangram_recording_clean.csv')
         .rename(columns={'utterance' : 'garbage', 'record' : 'utterance'})
         .query('utterance != "x"'))
d_raw = d_raw[pd.notnull(d_raw.utterance)]

In [ ]:
d_raw['text'] = [nlp(text) for text in d_raw['utterance']]

In [ ]:
d_raw['contentful'] = [[t.lemma_ for t in text if t.pos_ not in ['PRON', 'DET', 'CCONJ', 'ADP', 'AUX']] 
                       for text in d_raw['text']]

In [ ]:
null_embedding = np.full((1,300), np.nan)
def get_feats(d_in, nlp, scramble = False) :
    # only look at director utterances
    d = d_in.copy()

    # initialize feature vector
    raw_avg_feats = np.array([]).reshape(0, 300)

    if scramble :
        d = scramble_words(d)
        
    for i, row in d.iterrows() :
        local_embedding = np.array([]).reshape(0, 300)
        for token in row['contentful'] :
            if nlp(token).has_vector and sum(nlp(token).vector) != 0:
                local_embedding = np.vstack((local_embedding, nlp(token).vector))
            else :
                print(row['contentful'])
                print('no vector available:', nlp(token))

        # average them together, handling empty lists
        if row['contentful'] :
            raw_avg_embedding = np.nanmean(local_embedding, axis = 0) 
        else :
            
            raw_avg_embedding = null_embedding.copy()
            row['is_null'] = True
            
        # add to overall list
        raw_avg_feats = np.vstack((raw_avg_feats, raw_avg_embedding))
    return d, raw_avg_feats


In [ ]:
meta, raw_avg_feats = get_feats(d_raw, nlp)

In [ ]:
meta

In [ ]:
(meta
 .drop(columns = [ 'utterance', 'contentful'])
 .to_csv('../data/deidentified/meta_tangrams_embeddings.csv'))
np.save('../data/deidentified/feats_tangrams_embeddings_rawavg.npy', raw_avg_feats)

# Look at tsne visualization


In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.manifold import MDS
from sklearn.decomposition import TruncatedSVD
tsne = TSNE(n_components = 2)
big_pca = PCA(n_components = 40)
viz_pca = PCA(n_components = 2)
mds = MDS(n_components=2)

In [ ]:
embedding_viz = pd.DataFrame(
    columns = ['subid', 'target', 'trial', 'rep_num', 'x_tsne', 'y_tsne', 'x_mds', 'y_mds', 'feats_type']
)

for name, group in meta.reset_index(drop=True).groupby('target') :
    tangram_inds = np.array(group.index)
    feats = raw_avg_feats
    relevant_feats = feats[tangram_inds]
    
    # You can't run tsne with NANs, so we have to take them out and then add them back in...
    nan_rows = [i for i in range(relevant_feats.shape[0]) if pd.isna(relevant_feats[i,0])]
    nan_insert_rows = [k - lag for (lag, k) in enumerate(nan_rows)]
    X = np.ma.masked_invalid(relevant_feats)
    tsne_out = tsne.fit_transform(big_pca.fit_transform(np.ma.compress_rows(X)))
    tsne_out = np.insert(tsne_out, nan_insert_rows, np.nan, axis=0)
    X_tsne = pd.DataFrame(tsne_out, columns = ['x_tsne', 'y_tsne'], index=tangram_inds) 
    embedding_viz = embedding_viz.append(pd.concat([group, X_tsne], axis = 1), 
                                         ignore_index=True, sort=False)


In [ ]:
embedding_viz.drop(columns=['text', 'contentful']).to_csv('../data/deidentified/tsne_embeddings.csv')